In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
	.master("local[*]") \
	.appName('test') \
	.config("spark.executor.memory", "4g") \
	.config("spark.driver.memory", "4g") \
	.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/05 10:09:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df = spark.read \
    .options(header = "true", inferSchema = "true") \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv')

In [15]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', IntegerType(), True), StructField('DOlocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [5]:
!head data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv

dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,,B00009
B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,,B00013
B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264,264,,B00014
B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264,264,,B00014
B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264,264,,B00014
B00021         ,2019-10-01 00:00:48,2019-10-01 00:07:12,129,129,,B00021         
B00021         ,2019-10-01 00:47:23,2019-10-01 00:53:25,57,57,,B00021         
B00021         ,2019-10-01 00:10:06,2019-10-01 00:19:50,173,173,,B00021         
B00021         ,2019-10-01 00:51:37,2019-10-01 01:06:14,226,226,,B00021         


In [16]:
import pandas as pd

In [17]:
pd.DataFrame.iteritems = pd.DataFrame.items

In [30]:
df_pandas = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv')

In [31]:
df_pandas.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [32]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [33]:
spark.createDataFrame(df_pandas).schema

TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [34]:
from pyspark.sql import types

In [35]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
	types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [36]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv')

In [38]:
df = df.repartition(6)

In [39]:
df.write.parquet('fhv/2019/10/')

In [40]:
df = spark.read.parquet('fhv/2019/10/')

In [41]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [42]:
df.head(10)

[Row(dispatching_base_num='B00628', pickup_datetime=datetime.datetime(2019, 10, 4, 7, 6), dropoff_datetime=datetime.datetime(2019, 10, 4, 7, 38, 58), PULocationID=41, DOLocationID=265, SR_Flag=None, Affiliated_base_number='B02280'),
 Row(dispatching_base_num='B01639', pickup_datetime=datetime.datetime(2019, 10, 7, 16, 24, 31), dropoff_datetime=datetime.datetime(2019, 10, 7, 17, 2, 2), PULocationID=264, DOLocationID=127, SR_Flag=None, Affiliated_base_number='B01639'),
 Row(dispatching_base_num='B01315', pickup_datetime=datetime.datetime(2019, 10, 8, 13, 22, 9), dropoff_datetime=datetime.datetime(2019, 10, 8, 13, 47, 3), PULocationID=264, DOLocationID=159, SR_Flag=None, Affiliated_base_number='B01315'),
 Row(dispatching_base_num='B02401', pickup_datetime=datetime.datetime(2019, 10, 5, 13, 8, 16), dropoff_datetime=datetime.datetime(2019, 10, 5, 13, 23, 8), PULocationID=264, DOLocationID=17, SR_Flag=None, Affiliated_base_number='B02401'),
 Row(dispatching_base_num='B00111', pickup_datetime

In [43]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [45]:
df.createOrReplaceTempView('fhv_data')

In [59]:
spark.sql("""
SELECT
*
FROM
    fhv_data
where cast(pickup_datetime as date) = '2019-10-15'
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00254|2019-10-15 21:06:47|2019-10-15 21:17:43|          13|          68|   null|                B00254|
|              B00111|2019-10-15 06:18:17|2019-10-15 07:07:06|         264|         265|   null|                B00111|
|              B02849|2019-10-15 12:28:45|2019-10-15 12:59:40|         264|         145|   null|                B02849|
|              B01437|2019-10-15 11:46:44|2019-10-15 11:54:34|         264|         197|   null|                B01437|
|              B01437|2019-10-15 13:00:41|2019-10-15 13:35:06|         264|          82|   null|                B02880|
|              B02111|2019-10-15 22:43:0

In [58]:
spark.sql("""
SELECT
dispatching_base_num,
max(dropoff_datetime - pickup_datetime) as trip_distance 
FROM
    fhv_data
group by dispatching_base_num
order by trip_distance desc
""").show()

+--------------------+--------------------+
|dispatching_base_num|       trip_distance|
+--------------------+--------------------+
|              B02832|INTERVAL '26298 0...|
|              B02416|INTERVAL '3653 00...|
|     B00746         |INTERVAL '2922 00...|
|              B02921|INTERVAL '366 10:...|
|              B03110|INTERVAL '366 00:...|
|              B03080|INTERVAL '61 00:3...|
|     B03084         |INTERVAL '44 00:4...|
|              B01452|INTERVAL '33 01:3...|
|              B00972|INTERVAL '33 00:4...|
|              B02418|INTERVAL '31 01:3...|
|              B01455|INTERVAL '31 00:1...|
|              B02732|INTERVAL '25 04:0...|
|              B02532|INTERVAL '19 13:4...|
|              B02546|INTERVAL '18 00:4...|
|              B01985|INTERVAL '16 14:5...|
|              B03184|INTERVAL '14 19:0...|
|              B03107|INTERVAL '11 02:1...|
|              B02037|INTERVAL '10 00:1...|
|              B00272|INTERVAL '7 00:14...|
|              B02067|INTERVAL '

In [63]:
spark.sql("""
SELECT
(unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 as trip_distance 
FROM
    fhv_data
order by trip_distance desc
""").show()

+------------------+
|     trip_distance|
+------------------+
|          631152.5|
|          631152.5|
| 87672.44083333333|
| 70128.02805555555|
|            8794.0|
| 8784.166666666666|
|1464.5344444444445|
|1056.8266666666666|
|1056.2705555555556|
| 793.5530555555556|
| 793.3858333333334|
|          793.2975|
| 792.9980555555555|
| 792.9883333333333|
| 792.8602777777778|
| 792.8108333333333|
|           792.785|
| 792.7694444444444|
| 792.7538888888889|
| 792.7463888888889|
+------------------+
only showing top 20 rows



In [68]:
spark.sql("""
SELECT
 PULocationID, count(1)
FROM
    fhv_data
GROUP BY PULocationID
ORDER BY 2
""").show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|           2|       1|
|         105|       2|
|         111|       5|
|          30|       8|
|         120|      14|
|          12|      15|
|         207|      23|
|          27|      25|
|         154|      26|
|           8|      29|
|         128|      39|
|         253|      47|
|          96|      53|
|          34|      57|
|          59|      62|
|          58|      77|
|          99|      89|
|         190|      98|
|          54|     105|
|         217|     110|
+------------+--------+
only showing top 20 rows



In [69]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-03-05 14:09:03--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.169.32, 52.217.88.206, 52.217.116.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.169.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.08s   

2024-03-05 14:09:05 (148 KB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [70]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [73]:
df_zone.schema

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [74]:
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [75]:
df_zone = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('taxi+_zone_lookup.csv')

In [77]:
df_zone.write.parquet('zones')

In [78]:
df_zone = spark.read.parquet('zones')

In [82]:
df_zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [84]:
df_zone.head(10)

[Row(LocationID=1, Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID=2, Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID=3, Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID=4, Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID=5, Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone'),
 Row(LocationID=6, Borough='Staten Island', Zone='Arrochar/Fort Wadsworth', service_zone='Boro Zone'),
 Row(LocationID=7, Borough='Queens', Zone='Astoria', service_zone='Boro Zone'),
 Row(LocationID=8, Borough='Queens', Zone='Astoria Park', service_zone='Boro Zone'),
 Row(LocationID=9, Borough='Queens', Zone='Auburndale', service_zone='Boro Zone'),
 Row(LocationID=10, Borough='Queens', Zone='Baisley Park', service_zone='Boro Zone')]

In [86]:
df_zone.createOrReplaceTempView('zones')

In [87]:
spark.sql("""
SELECT
*
FROM
    zones
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [90]:
spark.sql("""
SELECT
	fhv.PULocationID, z.zone, count(1) 
FROM
    fhv_data fhv, zones z
where fhv.PULocationID = z.LocationID
GROUP BY fhv.PULocationID, z.zone
ORDER BY 3
""").show()

+------------+--------------------+--------+
|PULocationID|                zone|count(1)|
+------------+--------------------+--------+
|           2|         Jamaica Bay|       1|
|         105|Governor's Island...|       2|
|         111| Green-Wood Cemetery|       5|
|          30|       Broad Channel|       8|
|         120|     Highbridge Park|      14|
|          12|        Battery Park|      15|
|         207|Saint Michaels Ce...|      23|
|          27|Breezy Point/Fort...|      25|
|         154|Marine Park/Floyd...|      26|
|           8|        Astoria Park|      29|
|         128|    Inwood Hill Park|      39|
|         253|       Willets Point|      47|
|          96|Forest Park/Highl...|      53|
|          34|  Brooklyn Navy Yard|      57|
|          59|        Crotona Park|      62|
|          58|        Country Club|      77|
|          99|     Freshkills Park|      89|
|         190|       Prospect Park|      98|
|          54|     Columbia Street|     105|
|         